# Trabalho de IA

In [2]:
library('caret')

In [3]:
seed_universal <- 2022

## Funções gerais

### R2

In [199]:
r2 <- function(predito, observado) {
  return(1 - (sum((predito - observado) ^ 2) / sum((predito - mean(observado)) ^ 2)))
}

## Volume (Regressão)

In [200]:
volume.dados <- read.csv('bases/volume/Material 02 - 3 – Estimativa de Volume - Dados.csv')

In [201]:
summary(volume.dados)

          PROJETO    CLONE        Idade             Hdom         AreaBasal    
 Fazenda Velha:117   A2:119   Min.   : 21.30   Min.   : 8.80   Min.   : 3.20  
 Jacutinga    : 27   A3: 46   1st Qu.: 38.30   1st Qu.:19.80   1st Qu.:13.53  
 Parada       : 51   A4: 43   Median : 61.65   Median :26.00   Median :19.75  
 Santa Maria  :111   B7: 44   Mean   : 62.53   Mean   :24.38   Mean   :18.63  
                     C3: 54   3rd Qu.: 83.38   3rd Qu.:29.00   3rd Qu.:24.30  
                              Max.   :123.40   Max.   :35.30   Max.   :29.90  
     Volume     
 Min.   : 12.1  
 1st Qu.:221.2  
 Median :373.1  
 Mean   :384.6  
 3rd Qu.:548.6  
 Max.   :894.8  

In [202]:
head(volume.dados)

PROJETO,CLONE,Idade,Hdom,AreaBasal,Volume
Fazenda Velha,C3,21.5,8.8,3.2,12.1
Fazenda Velha,A3,21.9,10.2,3.9,66.8
Fazenda Velha,A3,26.1,10.1,4.2,72.1
Fazenda Velha,A4,22.0,10.0,4.2,35.9
Parada,B7,23.4,15.3,4.3,44.5
Fazenda Velha,A2,25.9,10.5,4.4,63.9


In [203]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(volume.dados$Volume, p = 0.8, list = FALSE)
treino <- volume.dados[indices,]
teste <- volume.dados[-indices,]

### Rede Neurais

#### Hold-out

In [41]:
set.seed(seed_universal)
rna <- train(Volume ~ ., data = treino, method = 'nnet', linout = T, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

246 samples
  5 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 246, 246, 246, 246, 246, 246, ... 
Resampling results across tuning parameters:

  size  decay  RMSE       Rsquared   MAE      
  1     0e+00  197.88119  0.4634644  168.81974
  1     1e-04  199.06606  0.2492164  170.46150
  1     1e-01  104.07515  0.6879326   86.55805
  3     0e+00  186.80061  0.3281662  159.29395
  3     1e-04  195.37767  0.2584346  166.93378
  3     1e-01   98.92831  0.7186257   79.25041
  5     0e+00  170.30300  0.4854395  142.69061
  5     1e-04  130.46402  0.5907056  108.58653
  5     1e-01   78.80306  0.8109754   62.54237

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 5 and decay = 0.1.

In [42]:
predicoes.rna <- predict(rna, teste)

In [43]:
r2(predicoes.rna, teste$Volume)

[1] 0.7954842

#### Cross validation

In [44]:
control <- trainControl(method = 'cv', number = 10)

set.seed(seed_universal)
rna <- train(Volume ~ ., data = treino, method = 'nnet', 
             linout = T, MaxNWts = 10000, maxit = 2000, trainControl = control, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

246 samples
  5 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 246, 246, 246, 246, 246, 246, ... 
Resampling results across tuning parameters:

  size  decay  RMSE       Rsquared   MAE       
  1     0e+00  197.88119  0.4634644  168.819740
  1     1e-04  199.06606  0.2492164  170.461499
  1     1e-01   43.95163  0.9088273   36.053049
  3     0e+00  186.75601  0.3282928  159.176976
  3     1e-04  192.49199  0.2844907  165.202780
  3     1e-01   16.44218  0.9768869   12.515733
  5     0e+00  169.22040  0.4919367  141.669076
  5     1e-04  121.78793  0.6183588  101.877294
  5     1e-01   10.71914  0.9970694    7.720742

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 5 and decay = 0.1.

In [45]:
predicoes.rna <- predict(rna, teste)

In [46]:
r2(predicoes.rna, teste$Volume)

[1] 0.9951281

##### Grid

In [47]:
control <- trainControl(method = 'cv', number = 10)
tuneGrid <- expand.grid(size = seq(from = 1, to = 10, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

set.seed(seed_universal)
rna <- train(Volume ~ ., data = treino, method = 'nnet', 
             trainControl = control, tuneGrid = tuneGrid, 
             linout = T, MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

246 samples
  5 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 246, 246, 246, 246, 246, 246, ... 
Resampling results across tuning parameters:

  size  decay  RMSE      Rsquared   MAE      
   1    0.1    65.30100  0.9329826  54.804849
   1    0.2    52.53010  0.9424187  43.261659
   1    0.3    39.94065  0.9182359  32.007308
   1    0.4    34.84986  0.9423415  27.399924
   1    0.5    43.35614  0.9802709  35.117258
   1    0.6    30.95972  0.9781187  23.581892
   1    0.7    45.17295  0.9384026  35.689355
   1    0.8    32.61827  0.9759435  24.729339
   1    0.9    40.34560  0.9423948  31.816750
   2    0.1    49.33668  0.8887616  41.129060
   2    0.2    17.20855  0.9922109  13.048608
   2    0.3    25.79076  0.9912054  20.020627
   2    0.4    17.03829  0.9930853  12.625986
   2    0.5    18.84305  0.9914069  13.699636
   2    0.6    19.64193  0.9907140  14.582907
   2    0.7    20.62063  0.9895238  15.043041
   2    0.8   

In [48]:
predicoes.rna <- predict(rna, teste)

In [49]:
r2(predicoes.rna, teste$Volume)

[1] 0.9936324

### KNN

In [206]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))
set.seed(seed_universal)
knn <- train(Volume ~ ., data = treino, method = "knn", btuneGrid=tuneGrid)
knn

k-Nearest Neighbors 

246 samples
  5 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 246, 246, 246, 246, 246, 246, ... 
Resampling results across tuning parameters:

  k  RMSE      Rsquared   MAE     
  5  108.0277  0.7167737  81.94722
  7  106.5912  0.7236263  80.65188
  9  105.7013  0.7276179  80.78625

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was k = 9.

In [209]:
predicoes.knn <- predict(knn, teste)
r2(predicoes.knn, teste$Volume)

[1] 0.5597709

----

## Biomassa (Regressão)

In [210]:
biomassa.dados <- read.csv('bases/biomassa/Material 02 - 4 - R - Biomassa - Dados.csv')

In [211]:
summary(biomassa.dados)

      dap               h               Me            biomassa       
 Min.   :  5.10   Min.   : 5.00   Min.   :0.2900   Min.   :    5.11  
 1st Qu.: 10.00   1st Qu.:11.50   1st Qu.:0.4975   1st Qu.:   35.87  
 Median : 14.00   Median :14.25   Median :0.6100   Median :   81.40  
 Mean   : 17.43   Mean   :15.40   Mean   :0.6216   Mean   :  398.94  
 3rd Qu.: 19.07   3rd Qu.:18.00   3rd Qu.:0.7200   3rd Qu.:  214.68  
 Max.   :138.00   Max.   :49.00   Max.   :1.0400   Max.   :18613.22  

In [212]:
head(biomassa.dados)

dap,h,Me,biomassa
6.4,5.0,1.04,7.07
7.3,5.0,1.04,10.30
7.8,5.5,1.04,13.90
9.2,7.6,1.04,18.61
9.9,8.2,1.04,30.97
10.6,8.7,1.04,46.44


In [213]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(biomassa.dados$biomassa, p = 0.8, list = FALSE)
treino <- biomassa.dados[indices,]
teste <- biomassa.dados[-indices,]

### Redes Neurais

In [78]:
set.seed(seed_universal)
rna <- train(biomassa ~ ., data = treino, method = 'nnet', linout = T, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

240 samples
  3 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 240, 240, 240, 240, 240, 240, ... 
Resampling results across tuning parameters:

  size  decay  RMSE      Rsquared   MAE     
  1     0e+00  1466.337  0.3644486  553.3771
  1     1e-04  1350.463  0.6292145  505.3208
  1     1e-01  1093.296  0.6891841  369.6148
  3     0e+00  1501.409  0.1433329  567.5949
  3     1e-04  1489.390  0.1699539  566.2313
  3     1e-01  1076.270  0.6121771  336.3104
  5     0e+00  1494.634  0.2168270  556.5481
  5     1e-04  1485.354  0.1807968  554.5376
  5     1e-01  1027.213  0.6594092  301.5608

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 5 and decay = 0.1.

In [79]:
predicoes.rna <- predict(rna, teste)

In [80]:
r2(predicoes.rna, teste$biomassa)

[1] 0.8578661

#### Cross validation

In [81]:
control <- trainControl(method = 'cv', number = 10)

set.seed(seed_universal)
rna <- train(biomassa ~ ., data = treino, method = 'nnet', 
             linout = T, MaxNWts = 10000, maxit = 2000, trainControl = control, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

240 samples
  3 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 240, 240, 240, 240, 240, 240, ... 
Resampling results across tuning parameters:

  size  decay  RMSE       Rsquared   MAE     
  1     0e+00  1466.3374  0.3644486  553.3771
  1     1e-04  1354.3819  0.6285031  505.8067
  1     1e-01  1089.2802  0.6880740  352.5654
  3     0e+00  1501.4088  0.1433329  567.5949
  3     1e-04  1489.3902  0.1699539  566.2313
  3     1e-01   898.5727  0.7973388  226.9956
  5     0e+00  1494.6336  0.2168270  556.5481
  5     1e-04  1485.3538  0.1807968  554.5376
  5     1e-01   816.1149  0.8411808  196.1931

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 5 and decay = 0.1.

In [82]:
predicoes.rna <- predict(rna, teste)

In [83]:
r2(predicoes.rna, teste$biomassa)

[1] 0.9161316

#### Grid

In [84]:
control <- trainControl(method = 'cv', number = 10)
tuneGrid <- expand.grid(size = seq(from = 1, to = 10, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

set.seed(seed_universal)
rna <- train(biomassa ~ ., data = treino, method = 'nnet', 
             trainControl = control, tuneGrid = tuneGrid, 
             linout = T, MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

240 samples
  3 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 240, 240, 240, 240, 240, 240, ... 
Resampling results across tuning parameters:

  size  decay  RMSE       Rsquared   MAE     
   1    0.1    1067.1821  0.6831073  353.2956
   1    0.2     980.8987  0.7018747  311.0923
   1    0.3     956.3362  0.8485133  310.5446
   1    0.4     962.6066  0.7492516  281.3446
   1    0.5    1103.8242  0.5923212  365.9093
   1    0.6     991.3940  0.7008012  305.7726
   1    0.7    1044.9901  0.6897010  317.1576
   1    0.8    1029.9935  0.7648785  315.6781
   1    0.9    1018.8826  0.7117438  299.8528
   2    0.1     952.2017  0.7758576  254.0573
   2    0.2     896.8212  0.7946368  225.7127
   2    0.3     886.3637  0.8239438  224.7773
   2    0.4     810.0870  0.8703751  204.5239
   2    0.5     888.7059  0.8321895  216.5902
   2    0.6     875.6846  0.8406566  229.3852
   2    0.7     893.9677  0.8289759  221.6518
   2    0.8   

In [85]:
predicoes.rna <- predict(rna, teste)

In [87]:
r2(predicoes.rna, teste$biomassa)

[1] 0.9132999

### KNN

In [214]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))
set.seed(seed_universal)
knn <- train(biomassa ~ ., data = treino, method = "knn", btuneGrid=tuneGrid)
knn

k-Nearest Neighbors 

240 samples
  3 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 240, 240, 240, 240, 240, 240, ... 
Resampling results across tuning parameters:

  k  RMSE      Rsquared   MAE     
  5  911.2977  0.7957026  188.7942
  7  939.6613  0.7826221  188.0198
  9  976.0060  0.7641584  193.1996

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was k = 5.

In [215]:
predicoes.knn <- predict(knn, teste)
r2(predicoes.knn, teste$biomassa)

[1] 0.8230908

----

## Admissão (Regressão)

In [216]:
admissao.dados <- read.csv('bases/admissao/Material 02 - 8 – R - Admissao - Dados.csv')

In [217]:
summary(admissao.dados)

   Serial.No.      GRE.Score      TOEFL.Score    University.Rating
 Min.   :  1.0   Min.   :290.0   Min.   : 92.0   Min.   :1.000    
 1st Qu.:125.8   1st Qu.:308.0   1st Qu.:103.0   1st Qu.:2.000    
 Median :250.5   Median :317.0   Median :107.0   Median :3.000    
 Mean   :250.5   Mean   :316.5   Mean   :107.2   Mean   :3.114    
 3rd Qu.:375.2   3rd Qu.:325.0   3rd Qu.:112.0   3rd Qu.:4.000    
 Max.   :500.0   Max.   :340.0   Max.   :120.0   Max.   :5.000    
      SOP             LOR             CGPA          Research   
 Min.   :1.000   Min.   :1.000   Min.   :6.800   Min.   :0.00  
 1st Qu.:2.500   1st Qu.:3.000   1st Qu.:8.127   1st Qu.:0.00  
 Median :3.500   Median :3.500   Median :8.560   Median :1.00  
 Mean   :3.374   Mean   :3.484   Mean   :8.576   Mean   :0.56  
 3rd Qu.:4.000   3rd Qu.:4.000   3rd Qu.:9.040   3rd Qu.:1.00  
 Max.   :5.000   Max.   :5.000   Max.   :9.920   Max.   :1.00  
 ChanceOfAdmit   
 Min.   :0.3400  
 1st Qu.:0.6300  
 Median :0.7200  
 Mean   :0.

In [218]:
head(admissao.dados)

Serial.No.,GRE.Score,TOEFL.Score,University.Rating,SOP,LOR,CGPA,Research,ChanceOfAdmit
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65
6,330,115,5,4.5,3.0,9.34,1,0.90


In [219]:
admissao.dados$Serial.No. <- NULL

In [220]:
head(admissao.dados)

GRE.Score,TOEFL.Score,University.Rating,SOP,LOR,CGPA,Research,ChanceOfAdmit
337,118,4,4.5,4.5,9.65,1,0.92
324,107,4,4.0,4.5,8.87,1,0.76
316,104,3,3.0,3.5,8.00,1,0.72
322,110,3,3.5,2.5,8.67,1,0.80
314,103,2,2.0,3.0,8.21,0,0.65
330,115,5,4.5,3.0,9.34,1,0.90


In [221]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(admissao.dados$ChanceOfAdmit, p = 0.8, list = FALSE)
treino <- admissao.dados[indices,]
teste <- admissao.dados[-indices,]

### Redes Neurais

In [94]:
set.seed(seed_universal)
rna <- train(ChanceOfAdmit ~ ., data = treino, method = 'nnet', linout = T, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

402 samples
  7 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 402, 402, 402, 402, 402, 402, ... 
Resampling results across tuning parameters:

  size  decay  RMSE        Rsquared   MAE       
  1     0e+00  0.14271606        NaN  0.11490328
  1     1e-04  0.13959213  0.4148616  0.11195705
  1     1e-01  0.07242726  0.7458084  0.05354218
  3     0e+00  0.13959972  0.7957222  0.11195485
  3     1e-04  0.09333584  0.6329480  0.07183258
  3     1e-01  0.07149190  0.7525573  0.05288111
  5     0e+00  0.13335357  0.8047199  0.10655677
  5     1e-04  0.08635205  0.6569240  0.06573398
  5     1e-01  0.07019944  0.7612076  0.05186431

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 5 and decay = 0.1.

In [95]:
predicoes.rna <- predict(rna, teste)

In [97]:
r2(predicoes.rna, teste$ChanceOfAdmit)

[1] 0.7367541

#### Cross validation

In [98]:
control <- trainControl(method = 'cv', number = 10)

set.seed(seed_universal)
rna <- train(ChanceOfAdmit ~ ., data = treino, method = 'nnet', 
             linout = T, MaxNWts = 10000, maxit = 2000, trainControl = control, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

402 samples
  7 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 402, 402, 402, 402, 402, 402, ... 
Resampling results across tuning parameters:

  size  decay  RMSE        Rsquared   MAE       
  1     0e+00  0.14271606        NaN  0.11490328
  1     1e-04  0.13959221  0.4148587  0.11195695
  1     1e-01  0.07242726  0.7458084  0.05354218
  3     0e+00  0.13959972  0.7957222  0.11195485
  3     1e-04  0.09775307  0.6063128  0.07297710
  3     1e-01  0.07101262  0.7559582  0.05236753
  5     0e+00  0.13337726  0.8036307  0.10656903
  5     1e-04  0.10533719  0.5815553  0.07113449
  5     1e-01  0.06940364  0.7663767  0.05079719

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 5 and decay = 0.1.

In [99]:
predicoes.rna <- predict(rna, teste)

In [100]:
r2(predicoes.rna, teste$ChanceOfAdmit)

[1] 0.7406736

#### Grid

In [101]:
control <- trainControl(method = 'cv', number = 10)
tuneGrid <- expand.grid(size = seq(from = 1, to = 10, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

set.seed(seed_universal)
rna <- train(ChanceOfAdmit ~ ., data = treino, method = 'nnet', 
             trainControl = control, tuneGrid = tuneGrid, 
             linout = T, MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Neural Network 

402 samples
  7 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 402, 402, 402, 402, 402, 402, ... 
Resampling results across tuning parameters:

  size  decay  RMSE        Rsquared   MAE       
   1    0.1    0.07240376  0.7460118  0.05351938
   1    0.2    0.07949528  0.6938832  0.05958088
   1    0.3    0.08087457  0.6829034  0.06086471
   1    0.4    0.08208196  0.6733589  0.06219753
   1    0.5    0.08203309  0.6738834  0.06212425
   1    0.6    0.08295717  0.6664795  0.06316990
   1    0.7    0.08327391  0.6651113  0.06356010
   1    0.8    0.08396669  0.6599031  0.06429069
   1    0.9    0.08429712  0.6579740  0.06458215
   2    0.1    0.07235634  0.7464313  0.05342085
   2    0.2    0.07795984  0.7055596  0.05834913
   2    0.3    0.07959692  0.6925372  0.05986246
   2    0.4    0.08039274  0.6864647  0.06054676
   2    0.5    0.08095644  0.6824760  0.06122800
   2    0.6    0.08181560  0.6763109  0.06213028
   2    0.7 

In [102]:
predicoes.rna <- predict(rna, teste)

In [103]:
r2(predicoes.rna, teste$ChanceOfAdmit)

[1] 0.7687059

### KNN

In [222]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))
set.seed(seed_universal)
knn <- train(ChanceOfAdmit ~ ., data = treino, method = "knn", btuneGrid=tuneGrid)
knn

k-Nearest Neighbors 

402 samples
  7 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 402, 402, 402, 402, 402, 402, ... 
Resampling results across tuning parameters:

  k  RMSE        Rsquared   MAE       
  5  0.07995354  0.6904343  0.05829223
  7  0.07701105  0.7096555  0.05639199
  9  0.07566495  0.7188611  0.05564908

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was k = 9.

In [223]:
predicoes.knn <- predict(knn, teste)
r2(predicoes.knn, teste$ChanceOfAdmit)

[1] 0.6629208

---

## Alunos do ensino médios (Regressão)

In [224]:
alunos.dados <- read.csv('bases/alunos_ensino_medio/Material 02 - 10 – Alunos - Dados.csv')

In [225]:
summary(alunos.dados)

 school        sex             age       address famsize   Pstatus
 GP:349   Min.   :1.000   Min.   :15.0   R: 88   GT3:281   A: 41  
 MS: 46   1st Qu.:1.000   1st Qu.:16.0   U:307   LE3:114   T:354  
          Median :2.000   Median :17.0                            
          Mean   :1.527   Mean   :16.7                            
          3rd Qu.:2.000   3rd Qu.:18.0                            
          Max.   :2.000   Max.   :22.0                            
      Medu            Fedu             Mjob           Fjob            reason   
 Min.   :0.000   Min.   :0.000   at_home : 59   at_home : 20   course    :145  
 1st Qu.:2.000   1st Qu.:2.000   health  : 34   health  : 18   home      :109  
 Median :3.000   Median :2.000   other   :141   other   :217   other     : 36  
 Mean   :2.749   Mean   :2.522   services:103   services:111   reputation:105  
 3rd Qu.:4.000   3rd Qu.:3.000   teacher : 58   teacher : 29                   
 Max.   :4.000   Max.   :4.000                     

In [226]:
head(alunos.dados)

school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
GP,2,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
GP,2,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
GP,2,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
GP,2,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
GP,2,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
GP,1,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15


In [227]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(alunos.dados$G3, p = 0.8, list = FALSE)
treino <- alunos.dados[indices,]
teste <- alunos.dados[-indices,]

### Redes Neurais

#### Hold out

In [109]:
set.seed(seed_universal)
rna <- train(G3 ~ ., data = treino, method = 'nnet', linout = T, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

318 samples
 32 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 318, 318, 318, 318, 318, 318, ... 
Resampling results across tuning parameters:

  size  decay  RMSE      Rsquared   MAE     
  1     0e+00  4.288185  0.4451051  3.290918
  1     1e-04  4.302246  0.4011125  3.291850
  1     1e-01  3.152601  0.5621009  2.461534
  3     0e+00  3.672778  0.4638319  2.809658
  3     1e-04  4.145325  0.3774091  3.208498
  3     1e-01  3.153018  0.5896584  2.416653
  5     0e+00  3.306683  0.5630039  2.507024
  5     1e-04  3.352291  0.5295603  2.536282
  5     1e-01  2.868583  0.6665035  2.097483

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 5 and decay = 0.1.

In [110]:
predicoes.rna <- predict(rna, teste)

In [112]:
r2(predicoes.rna, teste$G3)

[1] 0.6198192

#### Cross validation

In [113]:
control <- trainControl(method = 'cv', number = 10)

set.seed(seed_universal)
rna <- train(G3 ~ ., data = treino, method = 'nnet', 
             linout = T, MaxNWts = 10000, maxit = 2000, trainControl = control, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

Neural Network 

318 samples
 32 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 318, 318, 318, 318, 318, 318, ... 
Resampling results across tuning parameters:

  size  decay  RMSE      Rsquared   MAE     
  1     0e+00  4.166563  0.5201314  3.172987
  1     1e-04  4.031455  0.5510493  3.027069
  1     1e-01  2.379759  0.7465901  1.713231
  3     0e+00  3.437964  0.5298049  2.524120
  3     1e-04  3.852667  0.5037584  2.907719
  3     1e-01  2.924183  0.6707730  2.092594
  5     0e+00  3.200689  0.6106384  2.287803
  5     1e-04  3.746711  0.5264896  2.667822
  5     1e-01  2.819315  0.6881171  2.076416

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 1 and decay = 0.1.

In [114]:
predicoes.rna <- predict(rna, teste)

In [115]:
r2(predicoes.rna, teste$G3)

[1] 0.8049864

#### Grid

In [116]:
control <- trainControl(method = 'cv', number = 10)
tuneGrid <- expand.grid(size = seq(from = 1, to = 10, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

set.seed(seed_universal)
rna <- train(G3 ~ ., data = treino, method = 'nnet', 
             trainControl = control, tuneGrid = tuneGrid, 
             linout = T, MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Neural Network 

318 samples
 32 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 318, 318, 318, 318, 318, 318, ... 
Resampling results across tuning parameters:

  size  decay  RMSE      Rsquared   MAE     
   1    0.1    2.273496  0.7745711  1.624604
   1    0.2    2.293990  0.7701628  1.643472
   1    0.3    2.297142  0.7690740  1.645206
   1    0.4    2.309358  0.7661455  1.657495
   1    0.5    2.317427  0.7644550  1.664703
   1    0.6    2.336050  0.7602370  1.683033
   1    0.7    2.343881  0.7581943  1.694960
   1    0.8    2.350240  0.7569594  1.699965
   1    0.9    2.351879  0.7563162  1.704929
   2    0.1    2.650814  0.7152477  1.788418
   2    0.2    2.618882  0.7173510  1.776615
   2    0.3    2.532303  0.7322385  1.676403
   2    0.4    2.533874  0.7357208  1.695538
   2    0.5    2.588666  0.7218211  1.819977
   2    0.6    2.543715  0.7271131  1.732423
   2    0.7    2.616962  0.7134290  1.813050
   2    0.8    2.515453  0.7340

In [117]:
predicoes.rna <- predict(rna, teste)

In [118]:
r2(predicoes.rna, teste$G3)

[1] 0.8384283

### KNN

In [228]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))
set.seed(seed_universal)
knn <- train(G3 ~ ., data = treino, method = "knn", btuneGrid=tuneGrid)
knn

k-Nearest Neighbors 

318 samples
 32 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 318, 318, 318, 318, 318, 318, ... 
Resampling results across tuning parameters:

  k  RMSE      Rsquared   MAE     
  5  2.007207  0.8235914  1.366998
  7  2.012671  0.8255374  1.377899
  9  2.026420  0.8253318  1.397398

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was k = 5.

In [229]:
predicoes.knn <- predict(knn, teste)
r2(predicoes.knn, teste$G3)

[1] 0.8449737

---

## Câncer de Mama (Classificação)

In [138]:
cancer.dados <- read.csv('bases/cancer_mama/Material 02 - 2 - Cancer de Mama - Dados.csv')

In [139]:
summary(cancer.dados)

       Id         Cl.thickness      Cell.size        Cell.shape    
 Min.   :  1.0   Min.   : 1.000   Min.   : 1.000   Min.   : 1.000  
 1st Qu.:175.5   1st Qu.: 2.000   1st Qu.: 1.000   1st Qu.: 1.000  
 Median :350.0   Median : 4.000   Median : 1.000   Median : 1.000  
 Mean   :350.0   Mean   : 4.418   Mean   : 3.134   Mean   : 3.207  
 3rd Qu.:524.5   3rd Qu.: 6.000   3rd Qu.: 5.000   3rd Qu.: 5.000  
 Max.   :699.0   Max.   :10.000   Max.   :10.000   Max.   :10.000  
 Marg.adhesion     Epith.c.size     Bare.nuclei      Bl.cromatin    
 Min.   : 1.000   Min.   : 1.000   Min.   : 1.000   Min.   : 1.000  
 1st Qu.: 1.000   1st Qu.: 2.000   1st Qu.: 1.000   1st Qu.: 2.000  
 Median : 1.000   Median : 2.000   Median : 1.000   Median : 3.000  
 Mean   : 2.807   Mean   : 3.216   Mean   : 3.535   Mean   : 3.438  
 3rd Qu.: 4.000   3rd Qu.: 4.000   3rd Qu.: 6.000   3rd Qu.: 5.000  
 Max.   :10.000   Max.   :10.000   Max.   :10.000   Max.   :10.000  
 Normal.nucleoli     Mitoses             

In [142]:
head(cancer.dados)

Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
5,1,1,1,2,1,3,1,1,benign
5,4,4,5,7,10,3,2,1,benign
3,1,1,1,2,2,3,1,1,benign
6,8,8,1,3,4,3,7,1,benign
4,1,1,3,2,1,3,1,1,benign
8,10,10,8,7,10,9,7,1,malignant


In [143]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(cancer.dados$Class, p = 0.8, list = FALSE)
treino <- cancer.dados[indices,]
teste <- cancer.dados[-indices,]

### Redes Neurais

#### Hold out

In [54]:
set.seed(seed_universal)
rna <- train(Class ~ ., data = treino, method = 'nnet', trace = FALSE)
rna

Neural Network 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.9355606  0.8436904
  1     1e-04  0.8793081  0.6853898
  1     1e-01  0.9618024  0.9158194
  3     0e+00  0.9344482  0.8486237
  3     1e-04  0.9507364  0.8916345
  3     1e-01  0.9560586  0.9027121
  5     0e+00  0.9492720  0.8881370
  5     1e-04  0.9488739  0.8873937
  5     1e-01  0.9528808  0.8956393

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 1 and decay = 0.1.

In [55]:
predicoes.rna <- predict(rna, teste)

In [56]:
confusionMatrix(predicoes.rna, as.factor(teste$Class))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        88         2
  malignant      3        46
                                          
               Accuracy : 0.964           
                 95% CI : (0.9181, 0.9882)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9208          
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9670          
            Specificity : 0.9583          
         Pos Pred Value : 0.9778          
         Neg Pred Value : 0.9388          
             Prevalence : 0.6547          
         Detection Rate : 0.6331          
   Detection Prevalence : 0.6475          
      Balanced Accuracy : 0.9627          
                                          
       'Positive' Class : benign          

#### Cross validation

In [27]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)

In [28]:
set.seed(seed_universal)
rna <- train(Class ~ ., data = treino, method = 'nnet', trace = FALSE, trControl = ctrl)
rna

Neural Network 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 505, 505, 504, 504, 504, 503, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.9499294  0.8900761
  1     1e-04  0.9321372  0.8301133
  1     1e-01  0.9624943  0.9172935
  3     0e+00  0.9254956  0.8130474
  3     1e-04  0.9570734  0.9058046
  3     1e-01  0.9607411  0.9134859
  5     0e+00  0.9570734  0.9046412
  5     1e-04  0.9465493  0.8822215
  5     1e-01  0.9571058  0.9056348

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 1 and decay = 0.1.

In [30]:
predicoes.rna <- predict(rna, teste)

In [31]:
confusionMatrix(predicoes.rna, as.factor(teste$Class))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        88         2
  malignant      3        46
                                          
               Accuracy : 0.964           
                 95% CI : (0.9181, 0.9882)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9208          
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9670          
            Specificity : 0.9583          
         Pos Pred Value : 0.9778          
         Neg Pred Value : 0.9388          
             Prevalence : 0.6547          
         Detection Rate : 0.6331          
   Detection Prevalence : 0.6475          
      Balanced Accuracy : 0.9627          
                                          
       'Positive' Class : benign          

#### Grid

In [32]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)
grid <- expand.grid(size = seq(from = 1, to = 20, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

In [33]:
set.seed(seed_universal)
rna <- train(form = Class ~ .,data = treino,method = 'nnet',
             tuneGrid = grid,trControl = ctrl, MaxNWts = 10000, maxit = 2000, trace = FALSE)

In [36]:
rna

Neural Network 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 505, 505, 504, 504, 504, 503, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
   1    0.1    0.9624943  0.9172935
   1    0.2    0.9624943  0.9172935
   1    0.3    0.9660657  0.9252686
   1    0.4    0.9642162  0.9213477
   1    0.5    0.9642162  0.9213477
   1    0.6    0.9642487  0.9208951
   1    0.7    0.9642487  0.9208951
   1    0.8    0.9642487  0.9208951
   1    0.9    0.9642487  0.9209971
   2    0.1    0.9606448  0.9137523
   2    0.2    0.9607097  0.9133668
   2    0.3    0.9624305  0.9172071
   2    0.4    0.9624305  0.9172071
   2    0.5    0.9624630  0.9166443
   2    0.6    0.9624630  0.9166443
   2    0.7    0.9624630  0.9166443
   2    0.8    0.9624630  0.9166443
   2    0.9    0.9624630  0.9167463
   3    0.1    0.9571372  0.9051103
   3    0.2    0.9624305  0.9172071
 

In [37]:
predicoes.rna <- predict(rna, teste)

In [38]:
confusionMatrix(predicoes.rna, as.factor(teste$Class))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        88         4
  malignant      3        44
                                         
               Accuracy : 0.9496         
                 95% CI : (0.899, 0.9795)
    No Information Rate : 0.6547         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.8881         
                                         
 Mcnemar's Test P-Value : 1              
                                         
            Sensitivity : 0.9670         
            Specificity : 0.9167         
         Pos Pred Value : 0.9565         
         Neg Pred Value : 0.9362         
             Prevalence : 0.6547         
         Detection Rate : 0.6331         
   Detection Prevalence : 0.6619         
      Balanced Accuracy : 0.9418         
                                         
       'Positive' Class : benign         
                   

### KNN

In [148]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))

In [149]:
set.seed(seed_universal)
knn <- train(Class ~ ., data = treino, method = 'knn', tuneGrid = tuneGrid)
knn

k-Nearest Neighbors 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  k   Accuracy   Kappa    
   1  0.9530746  0.8962870
   3  0.9565261  0.9037733
   5  0.9588458  0.9089229
   7  0.9619745  0.9160224
   9  0.9619833  0.9158100
  11  0.9631333  0.9183482
  13  0.9631311  0.9184141
  15  0.9634177  0.9189507
  17  0.9633754  0.9188511
  19  0.9633914  0.9187926
  21  0.9628484  0.9175741

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 15.

In [152]:
predicoes.knn <- predict(knn, teste)
confusionMatrix(predicoes.knn, as.factor(teste$Class))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        88         3
  malignant      3        45
                                         
               Accuracy : 0.9568         
                 95% CI : (0.9084, 0.984)
    No Information Rate : 0.6547         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.9045         
                                         
 Mcnemar's Test P-Value : 1              
                                         
            Sensitivity : 0.9670         
            Specificity : 0.9375         
         Pos Pred Value : 0.9670         
         Neg Pred Value : 0.9375         
             Prevalence : 0.6547         
         Detection Rate : 0.6331         
   Detection Prevalence : 0.6547         
      Balanced Accuracy : 0.9523         
                                         
       'Positive' Class : benign         
                   

---

## Veículos (Classificação)

In [153]:
veiculos.dados <- read.csv('bases/veiculos/Material 02 - 5 - C - Veiculos - Dados.csv')

In [154]:
summary(veiculos.dados)

       a              Comp             Circ           DCirc       
 Min.   :  1.0   Min.   : 73.00   Min.   :33.00   Min.   : 40.00  
 1st Qu.:212.2   1st Qu.: 87.00   1st Qu.:40.00   1st Qu.: 70.00  
 Median :423.5   Median : 93.00   Median :44.00   Median : 80.00  
 Mean   :423.5   Mean   : 93.68   Mean   :44.86   Mean   : 82.09  
 3rd Qu.:634.8   3rd Qu.:100.00   3rd Qu.:49.00   3rd Qu.: 98.00  
 Max.   :846.0   Max.   :119.00   Max.   :59.00   Max.   :112.00  
     RadRa          PrAxisRa          MaxLRa           ScatRa     
 Min.   :104.0   Min.   : 47.00   Min.   : 2.000   Min.   :112.0  
 1st Qu.:141.0   1st Qu.: 57.00   1st Qu.: 7.000   1st Qu.:146.2  
 Median :167.0   Median : 61.00   Median : 8.000   Median :157.0  
 Mean   :168.9   Mean   : 61.69   Mean   : 8.567   Mean   :168.8  
 3rd Qu.:195.0   3rd Qu.: 65.00   3rd Qu.:10.000   3rd Qu.:198.0  
 Max.   :333.0   Max.   :138.00   Max.   :55.000   Max.   :265.0  
     Elong         PrAxisRect       MaxLRect     ScVarMaxis   

In [155]:
veiculos.dados$a <- NULL

In [156]:
head(veiculos.dados)

Comp,Circ,DCirc,RadRa,PrAxisRa,MaxLRa,ScatRa,Elong,PrAxisRect,MaxLRect,ScVarMaxis,ScVarmaxis,RaGyr,SkewMaxis,Skewmaxis,Kurtmaxis,KurtMaxis,HollRa,tipo
95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus
107,57,106,172,50,6,255,26,28,169,280,957,264,85,5,9,181,183,bus


In [157]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(veiculos.dados$tipo, p = 0.8, list = FALSE)
treino <- veiculos.dados[indices,]
teste <- veiculos.dados[-indices,]

### Redes Neurais

#### Hold out

In [62]:
set.seed(seed_universal)
rna <- train(tipo ~ ., data = treino, method = 'nnet', trace = FALSE)
rna

Neural Network 

679 samples
 18 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa     
  1     0e+00  0.2556064  0.02905632
  1     1e-04  0.2421816  0.01184603
  1     1e-01  0.4271395  0.24999845
  3     0e+00  0.2723895  0.05084082
  3     1e-04  0.2952822  0.07947773
  3     1e-01  0.5453347  0.39900893
  5     0e+00  0.3715668  0.17728169
  5     1e-04  0.2912832  0.07331797
  5     1e-01  0.5628079  0.42137699

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.1.

In [63]:
predicoes.rna <- predict(rna, teste)

In [64]:
confusionMatrix(predicoes.rna, as.factor(teste$tipo))

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   29    3    7   0
      opel   0    0    0   0
      saab   7   37   35   1
      van    7    2    1  38

Overall Statistics
                                          
               Accuracy : 0.6108          
                 95% CI : (0.5324, 0.6851)
    No Information Rate : 0.2575          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4806          
                                          
 Mcnemar's Test P-Value : 7.456e-09       

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.6744      0.0000      0.8140     0.9744
Specificity              0.9194      1.0000      0.6371     0.9219
Pos Pred Value           0.7436         NaN      0.4375     0.7917
Neg Pred Value           0.8906      0.7485      0.9080     0.9916
Prevalence               0.2575      

#### Cross validation

In [65]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)

In [66]:
set.seed(seed_universal)
rna <- train(tipo ~ ., data = treino, method = 'nnet', trace = FALSE, trControl = ctrl)
rna

Neural Network 

679 samples
 18 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 612, 611, 611, 611, 611, 610, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa     
  1     0e+00  0.2845049  0.03738740
  1     1e-04  0.2771739  0.02949643
  1     1e-01  0.4459381  0.26310192
  3     0e+00  0.2830789  0.03738993
  3     1e-04  0.3621690  0.14475717
  3     1e-01  0.5738914  0.43195317
  5     0e+00  0.4641244  0.28324896
  5     1e-04  0.3566607  0.13885904
  5     1e-01  0.6357722  0.51415590

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.1.

In [67]:
predicoes.rna <- predict(rna, teste)

In [68]:
confusionMatrix(predicoes.rna, as.factor(teste$tipo))

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   32    0    2  13
      opel   0    4    4   0
      saab   9   36   33   0
      van    2    2    4  26

Overall Statistics
                                          
               Accuracy : 0.5689          
                 95% CI : (0.4901, 0.6451)
    No Information Rate : 0.2575          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4234          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.7442     0.09524      0.7674     0.6667
Specificity              0.8790     0.96800      0.6371     0.9375
Pos Pred Value           0.6809     0.50000      0.4231     0.7647
Neg Pred Value           0.9083     0.76101      0.8876     0.9023
Prevalence               0.2575     0

#### Grid

In [69]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)
grid <- expand.grid(size = seq(from = 1, to = 20, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

In [70]:
set.seed(seed_universal)
rna <- train(form = tipo ~ .,data = treino, method = 'nnet',
             tuneGrid = grid, trControl = ctrl, MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Neural Network 

679 samples
 18 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 612, 611, 611, 611, 611, 610, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
   1    0.1    0.4955141  0.3255716
   1    0.2    0.4936787  0.3214073
   1    0.3    0.5089867  0.3413993
   1    0.4    0.5523536  0.4025164
   1    0.5    0.5184585  0.3560281
   1    0.6    0.5392030  0.3843746
   1    0.7    0.5523950  0.4005104
   1    0.8    0.5243434  0.3638023
   1    0.9    0.5230672  0.3619274
   2    0.1    0.7127947  0.6165796
   2    0.2    0.7304431  0.6398160
   2    0.3    0.7246246  0.6320257
   2    0.4    0.7202342  0.6260461
   2    0.5    0.7155823  0.6202173
   2    0.6    0.7320673  0.6425441
   2    0.7    0.7217048  0.6282929
   2    0.8    0.7029081  0.6029649
   2    0.9    0.7246027  0.6318869
   3    0.1    0.7878895  0.7168167
   3    0.2    0.7702606  0.69

In [71]:
predicoes.rna <- predict(rna, teste)

In [72]:
confusionMatrix(predicoes.rna, as.factor(teste$tipo))

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   40    2    4   3
      opel   2   25   10   0
      saab   0   13   29   1
      van    1    2    0  35

Overall Statistics
                                          
               Accuracy : 0.7725          
                 95% CI : (0.7013, 0.8337)
    No Information Rate : 0.2575          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.6963          
                                          
 Mcnemar's Test P-Value : 0.2108          

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.9302      0.5952      0.6744     0.8974
Specificity              0.9274      0.9040      0.8871     0.9766
Pos Pred Value           0.8163      0.6757      0.6744     0.9211
Neg Pred Value           0.9746      0.8692      0.8871     0.9690
Prevalence               0.2575      

### KNN

In [158]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))

In [159]:
set.seed(seed_universal)
knn <- train(tipo ~ ., data = treino, method = 'knn', tuneGrid = tuneGrid)
knn

k-Nearest Neighbors 

679 samples
 18 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  k   Accuracy   Kappa    
   1  0.6219105  0.4954732
   3  0.6014193  0.4680911
   5  0.6080492  0.4774091
   7  0.6045153  0.4726698
   9  0.5994078  0.4660012
  11  0.5935021  0.4583123
  13  0.5934363  0.4584592
  15  0.5867011  0.4496481
  17  0.5830713  0.4449385
  19  0.5836042  0.4455362
  21  0.5786242  0.4391090

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 1.

In [160]:
predicoes.knn <- predict(knn, teste)
confusionMatrix(predicoes.knn, as.factor(teste$tipo))

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   36    2    5   4
      opel   2   19   15   1
      saab   2   17   22   1
      van    3    4    1  33

Overall Statistics
                                          
               Accuracy : 0.6587          
                 95% CI : (0.5814, 0.7302)
    No Information Rate : 0.2575          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.5447          
                                          
 Mcnemar's Test P-Value : 0.7633          

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.8372      0.4524      0.5116     0.8462
Specificity              0.9113      0.8560      0.8387     0.9375
Pos Pred Value           0.7660      0.5135      0.5238     0.8049
Neg Pred Value           0.9417      0.8231      0.8320     0.9524
Prevalence               0.2575      

----

## Previsão do tempo (Classificação)

In [161]:
previsao.dados <- read.csv('bases/previsao_tempo/Material 02 - 6 - C - Previsao do Tempo - Dados.csv')

In [162]:
summary(previsao.dados)

         Ceu      Temperatura   Umidade   Vento    Chovera 
 Chuvoso   :16   Baixa  :16   Alta  :20   Nao:28   NAO:24  
 Coberto   : 8   Elevada:12   Normal:20   Sim:12   SIM:16  
 Ensolarado:16   Media  :12                                

In [163]:
head(previsao.dados)

Ceu,Temperatura,Umidade,Vento,Chovera
Ensolarado,Elevada,Alta,Nao,SIM
Ensolarado,Elevada,Alta,Sim,SIM
Coberto,Elevada,Alta,Nao,NAO
Chuvoso,Media,Alta,Nao,NAO
Chuvoso,Baixa,Normal,Nao,NAO
Chuvoso,Baixa,Normal,Sim,SIM


In [164]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(previsao.dados$Chovera, p = 0.8, list = FALSE)
treino <- previsao.dados[indices,]
teste <- previsao.dados[-indices,]

### Redes Neurais

#### Hold out

In [77]:
set.seed(seed_universal)
rna <- train(Chovera ~ ., data = treino, method = 'nnet', trace = FALSE)
rna

Neural Network 

33 samples
 4 predictor
 2 classes: 'NAO', 'SIM' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 33, 33, 33, 33, 33, 33, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.9167386  0.8306648
  1     1e-04  0.9335744  0.8609592
  1     1e-01  0.9171691  0.8329419
  3     0e+00  0.9525195  0.9021314
  3     1e-04  0.9525195  0.9021314
  3     1e-01  0.9251691  0.8477982
  5     0e+00  0.9490909  0.8967980
  5     1e-04  0.9525195  0.9021314
  5     1e-01  0.9171691  0.8329419

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 3 and decay = 1e-04.

In [78]:
predicoes.rna <- predict(rna, teste)

In [79]:
confusionMatrix(predicoes.rna, as.factor(teste$Chovera))

Confusion Matrix and Statistics

          Reference
Prediction NAO SIM
       NAO   4   0
       SIM   0   3
                                     
               Accuracy : 1          
                 95% CI : (0.5904, 1)
    No Information Rate : 0.5714     
    P-Value [Acc > NIR] : 0.01989    
                                     
                  Kappa : 1          
                                     
 Mcnemar's Test P-Value : NA         
                                     
            Sensitivity : 1.0000     
            Specificity : 1.0000     
         Pos Pred Value : 1.0000     
         Neg Pred Value : 1.0000     
             Prevalence : 0.5714     
         Detection Rate : 0.5714     
   Detection Prevalence : 0.5714     
      Balanced Accuracy : 1.0000     
                                     
       'Positive' Class : NAO        
                                     

#### Cross validation

In [84]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)

In [85]:
set.seed(seed_universal)
rna <- train(Chovera ~ ., data = treino, method = 'nnet', trace = FALSE, trControl = ctrl)
rna

Neural Network 

33 samples
 4 predictor
 2 classes: 'NAO', 'SIM' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 30, 30, 30, 30, 30, 30, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy  Kappa
  1     0e+00  1         1    
  1     1e-04  1         1    
  1     1e-01  1         1    
  3     0e+00  1         1    
  3     1e-04  1         1    
  3     1e-01  1         1    
  5     0e+00  1         1    
  5     1e-04  1         1    
  5     1e-01  1         1    

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 1 and decay = 0.1.

In [86]:
predicoes.rna <- predict(rna, teste)

In [87]:
confusionMatrix(predicoes.rna, as.factor(teste$Chovera))

Confusion Matrix and Statistics

          Reference
Prediction NAO SIM
       NAO   4   0
       SIM   0   3
                                     
               Accuracy : 1          
                 95% CI : (0.5904, 1)
    No Information Rate : 0.5714     
    P-Value [Acc > NIR] : 0.01989    
                                     
                  Kappa : 1          
                                     
 Mcnemar's Test P-Value : NA         
                                     
            Sensitivity : 1.0000     
            Specificity : 1.0000     
         Pos Pred Value : 1.0000     
         Neg Pred Value : 1.0000     
             Prevalence : 0.5714     
         Detection Rate : 0.5714     
   Detection Prevalence : 0.5714     
      Balanced Accuracy : 1.0000     
                                     
       'Positive' Class : NAO        
                                     

#### Grid

In [88]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)
grid <- expand.grid(size = seq(from = 1, to = 20, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

In [89]:
set.seed(seed_universal)
rna <- train(form = Chovera ~ .,data = treino, method = 'nnet',
             tuneGrid = grid, trControl = ctrl, MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Neural Network 

33 samples
 4 predictor
 2 classes: 'NAO', 'SIM' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 30, 30, 30, 30, 30, 30, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa
   1    0.1    1.0000000  1.00 
   1    0.2    1.0000000  1.00 
   1    0.3    0.8333333  0.60 
   1    0.4    0.7750000  0.45 
   1    0.5    0.7083333  0.25 
   1    0.6    0.6166667  0.00 
   1    0.7    0.6166667  0.00 
   1    0.8    0.6166667  0.00 
   1    0.9    0.6166667  0.00 
   2    0.1    1.0000000  1.00 
   2    0.2    0.9666667  0.90 
   2    0.3    0.8666667  0.70 
   2    0.4    0.8333333  0.60 
   2    0.5    0.7666667  0.40 
   2    0.6    0.6166667  0.00 
   2    0.7    0.6166667  0.00 
   2    0.8    0.6166667  0.00 
   2    0.9    0.6166667  0.00 
   3    0.1    1.0000000  1.00 
   3    0.2    0.9750000  0.95 
   3    0.3    0.8333333  0.60 
   3    0.4    0.8333333  0.60 
   3    0.5    0.8000000  0.50 
   3   

In [90]:
predicoes.rna <- predict(rna, teste)

In [91]:
confusionMatrix(predicoes.rna, as.factor(teste$Chovera))

Confusion Matrix and Statistics

          Reference
Prediction NAO SIM
       NAO   4   0
       SIM   0   3
                                     
               Accuracy : 1          
                 95% CI : (0.5904, 1)
    No Information Rate : 0.5714     
    P-Value [Acc > NIR] : 0.01989    
                                     
                  Kappa : 1          
                                     
 Mcnemar's Test P-Value : NA         
                                     
            Sensitivity : 1.0000     
            Specificity : 1.0000     
         Pos Pred Value : 1.0000     
         Neg Pred Value : 1.0000     
             Prevalence : 0.5714     
         Detection Rate : 0.5714     
   Detection Prevalence : 0.5714     
      Balanced Accuracy : 1.0000     
                                     
       'Positive' Class : NAO        
                                     

### KNN

In [165]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))

In [166]:
set.seed(seed_universal)
knn <- train(Chovera ~ ., data = treino, method = 'knn', tuneGrid = tuneGrid)
knn

k-Nearest Neighbors 

33 samples
 4 predictor
 2 classes: 'NAO', 'SIM' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 33, 33, 33, 33, 33, 33, ... 
Resampling results across tuning parameters:

  k   Accuracy   Kappa    
   1  0.9596923  0.9181141
   3  0.8215593  0.6391019
   5  0.7076303  0.4117436
   7  0.6648560  0.3319262
   9  0.6690882  0.3394840
  11  0.6847934  0.3640808
  13  0.6674981  0.3119448
  15  0.6587015  0.2854068
  17  0.6190865  0.2051913
  19  0.6029086  0.1748818
  21  0.5845741  0.1299148

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 1.

In [167]:
predicoes.knn <- predict(knn, teste)
confusionMatrix(predicoes.knn, as.factor(teste$Chovera))

Confusion Matrix and Statistics

          Reference
Prediction NAO SIM
       NAO   4   0
       SIM   0   3
                                     
               Accuracy : 1          
                 95% CI : (0.5904, 1)
    No Information Rate : 0.5714     
    P-Value [Acc > NIR] : 0.01989    
                                     
                  Kappa : 1          
                                     
 Mcnemar's Test P-Value : NA         
                                     
            Sensitivity : 1.0000     
            Specificity : 1.0000     
         Pos Pred Value : 1.0000     
         Neg Pred Value : 1.0000     
             Prevalence : 0.5714     
         Detection Rate : 0.5714     
   Detection Prevalence : 0.5714     
      Balanced Accuracy : 1.0000     
                                     
       'Positive' Class : NAO        
                                     

---

## IR (Classifição)

In [168]:
ir.dados <- read.csv('bases/imposto_renda/Material 02 - 7 – C - IR - Dados.csv')

In [169]:
summary(ir.dados)

  rest           ecivil     rendimento      sonegador 
 Nao:35   Casado    :20   Min.   : 60000   Nao   :20  
 Sim:15   Divorciado:10   1st Qu.: 75000   Sim   :15  
          Solteiro  :20   Median : 92500   Talvez:15  
                          Mean   :104000              
                          3rd Qu.:120000              
                          Max.   :220000              

In [170]:
head(ir.dados)

rest,ecivil,rendimento,sonegador
Sim,Solteiro,125000,Sim
Nao,Casado,100000,Nao
Nao,Solteiro,70000,Talvez
Sim,Casado,120000,Sim
Nao,Divorciado,95000,Talvez
Nao,Casado,60000,Nao


In [171]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(ir.dados$sonegador, p = 0.8, list = FALSE)
treino <- ir.dados[indices,]
teste <- ir.dados[-indices,]

### Redes Neurais

#### Hold out

In [96]:
set.seed(seed_universal)
rna <- train(sonegador ~ ., data = treino, method = 'nnet', trace = FALSE)
rna

Neural Network 

40 samples
 3 predictor
 3 classes: 'Nao', 'Sim', 'Talvez' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 40, 40, 40, 40, 40, 40, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa       
  1     0e+00  0.2969527   0.004444444
  1     1e-04  0.2869527   0.000000000
  1     1e-01  0.4337769   0.151887306
  3     0e+00  0.2919527   0.000000000
  3     1e-04  0.2844527  -0.008372093
  3     1e-01  0.5748208   0.336738077
  5     0e+00  0.2869527  -0.006829268
  5     1e-04  0.3182860   0.023892617
  5     1e-01  0.5835300   0.365057930

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.1.

In [97]:
predicoes.rna <- predict(rna, teste)

In [98]:
confusionMatrix(predicoes.rna, as.factor(teste$sonegador))

Confusion Matrix and Statistics

          Reference
Prediction Nao Sim Talvez
    Nao      4   0      0
    Sim      0   2      0
    Talvez   0   1      3

Overall Statistics
                                         
               Accuracy : 0.9            
                 95% CI : (0.555, 0.9975)
    No Information Rate : 0.4            
    P-Value [Acc > NIR] : 0.001678       
                                         
                  Kappa : 0.8485         
                                         
 Mcnemar's Test P-Value : NA             

Statistics by Class:

                     Class: Nao Class: Sim Class: Talvez
Sensitivity                 1.0     0.6667        1.0000
Specificity                 1.0     1.0000        0.8571
Pos Pred Value              1.0     1.0000        0.7500
Neg Pred Value              1.0     0.8750        1.0000
Prevalence                  0.4     0.3000        0.3000
Detection Rate              0.4     0.2000        0.3000
Detection Prevalence   

#### Cross validation

In [99]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)

In [100]:
set.seed(seed_universal)
rna <- train(sonegador ~ ., data = treino, method = 'nnet', trace = FALSE, trControl = ctrl)
rna

Neural Network 

40 samples
 3 predictor
 3 classes: 'Nao', 'Sim', 'Talvez' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 37, 36, 35, 36, 37, 35, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.3950000  0.0000000
  1     1e-04  0.3950000  0.0000000
  1     1e-01  0.4283333  0.0500000
  3     0e+00  0.3950000  0.0000000
  3     1e-04  0.3950000  0.0000000
  3     1e-01  0.5266667  0.2377778
  5     0e+00  0.3950000  0.0000000
  5     1e-04  0.3950000  0.0000000
  5     1e-01  0.6133333  0.4100000

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.1.

In [101]:
predicoes.rna <- predict(rna, teste)

In [102]:
confusionMatrix(predicoes.rna, as.factor(teste$sonegador))

Confusion Matrix and Statistics

          Reference
Prediction Nao Sim Talvez
    Nao      4   3      3
    Sim      0   0      0
    Talvez   0   0      0

Overall Statistics
                                          
               Accuracy : 0.4             
                 95% CI : (0.1216, 0.7376)
    No Information Rate : 0.4             
    P-Value [Acc > NIR] : 0.6177          
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: Nao Class: Sim Class: Talvez
Sensitivity                 1.0        0.0           0.0
Specificity                 0.0        1.0           1.0
Pos Pred Value              0.4        NaN           NaN
Neg Pred Value              NaN        0.7           0.7
Prevalence                  0.4        0.3           0.3
Detection Rate              0.4        0.0           0.0
Detection Prev

#### Grid

In [103]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)
grid <- expand.grid(size = seq(from = 1, to = 20, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

In [104]:
set.seed(seed_universal)
rna <- train(form = sonegador ~ .,data = treino, method = 'nnet',
             tuneGrid = grid, trControl = ctrl, MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Neural Network 

40 samples
 3 predictor
 3 classes: 'Nao', 'Sim', 'Talvez' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 37, 36, 35, 36, 37, 35, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa        
   1    0.1    0.4650000   1.011765e-01
   1    0.2    0.4600000   1.222222e-01
   1    0.3    0.3950000   0.000000e+00
   1    0.4    0.4600000   1.342320e-01
   1    0.5    0.3950000   0.000000e+00
   1    0.6    0.3950000  -9.251859e-18
   1    0.7    0.3950000   0.000000e+00
   1    0.8    0.3950000   0.000000e+00
   1    0.9    0.3950000   0.000000e+00
   2    0.1    0.5016667   1.666667e-01
   2    0.2    0.5250000   2.333333e-01
   2    0.3    0.5566667   2.411765e-01
   2    0.4    0.5350000   2.576389e-01
   2    0.5    0.4533333   1.118056e-01
   2    0.6    0.4350000   8.492647e-02
   2    0.7    0.3950000   0.000000e+00
   2    0.8    0.3950000   0.000000e+00
   2    0.9    0.3950000   0.000000e+00
   3 

In [105]:
predicoes.rna <- predict(rna, teste)

In [106]:
confusionMatrix(predicoes.rna, as.factor(teste$sonegador))

Confusion Matrix and Statistics

          Reference
Prediction Nao Sim Talvez
    Nao      4   0      0
    Sim      0   2      0
    Talvez   0   1      3

Overall Statistics
                                         
               Accuracy : 0.9            
                 95% CI : (0.555, 0.9975)
    No Information Rate : 0.4            
    P-Value [Acc > NIR] : 0.001678       
                                         
                  Kappa : 0.8485         
                                         
 Mcnemar's Test P-Value : NA             

Statistics by Class:

                     Class: Nao Class: Sim Class: Talvez
Sensitivity                 1.0     0.6667        1.0000
Specificity                 1.0     1.0000        0.8571
Pos Pred Value              1.0     1.0000        0.7500
Neg Pred Value              1.0     0.8750        1.0000
Prevalence                  0.4     0.3000        0.3000
Detection Rate              0.4     0.2000        0.3000
Detection Prevalence   

### KNN

In [172]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))

In [173]:
set.seed(seed_universal)
knn <- train(sonegador ~ ., data = treino, method = 'knn', tuneGrid = tuneGrid)
knn

k-Nearest Neighbors 

40 samples
 3 predictor
 3 classes: 'Nao', 'Sim', 'Talvez' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 40, 40, 40, 40, 40, 40, ... 
Resampling results across tuning parameters:

  k   Accuracy   Kappa    
   1  0.9796923  0.9694232
   3  0.7314928  0.6005095
   5  0.5784559  0.3825846
   7  0.5270330  0.3021497
   9  0.5009569  0.2845151
  11  0.4980954  0.2791060
  13  0.4668500  0.2448484
  15  0.4214745  0.1793150
  17  0.4100643  0.1673195
  19  0.3848200  0.1403709
  21  0.3408785  0.1049611

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 1.

In [174]:
predicoes.knn <- predict(knn, teste)
confusionMatrix(predicoes.knn, as.factor(teste$sonegador))

Confusion Matrix and Statistics

          Reference
Prediction Nao Sim Talvez
    Nao      4   0      0
    Sim      0   3      0
    Talvez   0   0      3

Overall Statistics
                                     
               Accuracy : 1          
                 95% CI : (0.6915, 1)
    No Information Rate : 0.4        
    P-Value [Acc > NIR] : 0.0001049  
                                     
                  Kappa : 1          
                                     
 Mcnemar's Test P-Value : NA         

Statistics by Class:

                     Class: Nao Class: Sim Class: Talvez
Sensitivity                 1.0        1.0           1.0
Specificity                 1.0        1.0           1.0
Pos Pred Value              1.0        1.0           1.0
Neg Pred Value              1.0        1.0           1.0
Prevalence                  0.4        0.3           0.3
Detection Rate              0.4        0.3           0.3
Detection Prevalence        0.4        0.3           0.3
Ba

___

## Diabetes (Classificação)

In [176]:
diabetes.dados <- read.csv('bases/diabetes/Material 02 - 9 – C - Diabetes - Dados.csv')

In [177]:
summary(diabetes.dados)

      num           preg0nt          glucose         pressure     
 Min.   :  1.0   Min.   : 0.000   Min.   :  0.0   Min.   :  0.00  
 1st Qu.:192.8   1st Qu.: 1.000   1st Qu.: 99.0   1st Qu.: 62.00  
 Median :384.5   Median : 3.000   Median :117.0   Median : 72.00  
 Mean   :384.5   Mean   : 3.845   Mean   :120.9   Mean   : 69.11  
 3rd Qu.:576.2   3rd Qu.: 6.000   3rd Qu.:140.2   3rd Qu.: 80.00  
 Max.   :768.0   Max.   :17.000   Max.   :199.0   Max.   :122.00  
    triceps         insulin           mass          pedigree     
 Min.   : 0.00   Min.   :  0.0   Min.   : 0.00   Min.   :0.0780  
 1st Qu.: 0.00   1st Qu.:  0.0   1st Qu.:27.30   1st Qu.:0.2437  
 Median :23.00   Median : 30.5   Median :32.00   Median :0.3725  
 Mean   :20.54   Mean   : 79.8   Mean   :31.99   Mean   :0.4719  
 3rd Qu.:32.00   3rd Qu.:127.2   3rd Qu.:36.60   3rd Qu.:0.6262  
 Max.   :99.00   Max.   :846.0   Max.   :67.10   Max.   :2.4200  
      age        diabetes 
 Min.   :21.00   neg:500  
 1st Qu.:24.00 

In [178]:
diabetes.dados$num <- NULL

In [179]:
head(diabetes.dados)

preg0nt,glucose,pressure,triceps,insulin,mass,pedigree,age,diabetes
6,148,72,35,0,33.6,0.627,50,pos
1,85,66,29,0,26.6,0.351,31,neg
8,183,64,0,0,23.3,0.672,32,pos
1,89,66,23,94,28.1,0.167,21,neg
0,137,40,35,168,43.1,2.288,33,pos
5,116,74,0,0,25.6,0.201,30,neg


In [180]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(diabetes.dados$diabetes, p = 0.8, list = FALSE)
treino <- diabetes.dados[indices,]
teste <- diabetes.dados[-indices,]

### Redes Neurais

#### Hold out

In [112]:
set.seed(seed_universal)
rna <- train(diabetes ~ ., data = treino, method = 'nnet', trace = FALSE)
rna

Neural Network 

615 samples
  8 predictor
  2 classes: 'neg', 'pos' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 615, 615, 615, 615, 615, 615, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa     
  1     0e+00  0.6462388  0.05218711
  1     1e-04  0.6445605  0.06087513
  1     1e-01  0.6439753  0.14101926
  3     0e+00  0.6430005  0.12656845
  3     1e-04  0.6442854  0.09488189
  3     1e-01  0.6794833  0.25046095
  5     0e+00  0.6572974  0.18220200
  5     1e-04  0.6578923  0.18440297
  5     1e-01  0.6854652  0.26966042

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.1.

In [113]:
predicoes.rna <- predict(rna, teste)

In [114]:
confusionMatrix(predicoes.rna, as.factor(teste$diabetes))

Confusion Matrix and Statistics

          Reference
Prediction neg pos
       neg  81  24
       pos  19  29
                                          
               Accuracy : 0.719           
                 95% CI : (0.6407, 0.7886)
    No Information Rate : 0.6536          
    P-Value [Acc > NIR] : 0.05152         
                                          
                  Kappa : 0.3653          
                                          
 Mcnemar's Test P-Value : 0.54187         
                                          
            Sensitivity : 0.8100          
            Specificity : 0.5472          
         Pos Pred Value : 0.7714          
         Neg Pred Value : 0.6042          
             Prevalence : 0.6536          
         Detection Rate : 0.5294          
   Detection Prevalence : 0.6863          
      Balanced Accuracy : 0.6786          
                                          
       'Positive' Class : neg             
                              

#### Cross validation

In [115]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)

In [116]:
set.seed(seed_universal)
rna <- train(diabetes ~ ., data = treino, method = 'nnet', trace = FALSE, trControl = ctrl)
rna

Neural Network 

615 samples
  8 predictor
  2 classes: 'neg', 'pos' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 554, 554, 554, 554, 553, 553, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa      
  1     0e+00  0.6276838  0.007969641
  1     1e-04  0.6536753  0.090645527
  1     1e-01  0.7447911  0.402348287
  3     0e+00  0.6309889  0.084825931
  3     1e-04  0.6766526  0.126146092
  3     1e-01  0.7544685  0.451399338
  5     0e+00  0.6488102  0.122877574
  5     1e-04  0.7332893  0.377536533
  5     1e-01  0.7059228  0.322349057

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 3 and decay = 0.1.

In [117]:
predicoes.rna <- predict(rna, teste)

In [118]:
confusionMatrix(predicoes.rna, as.factor(teste$diabetes))

Confusion Matrix and Statistics

          Reference
Prediction neg pos
       neg  84  21
       pos  16  32
                                          
               Accuracy : 0.7582          
                 95% CI : (0.6824, 0.8237)
    No Information Rate : 0.6536          
    P-Value [Acc > NIR] : 0.003479        
                                          
                  Kappa : 0.4538          
                                          
 Mcnemar's Test P-Value : 0.510798        
                                          
            Sensitivity : 0.8400          
            Specificity : 0.6038          
         Pos Pred Value : 0.8000          
         Neg Pred Value : 0.6667          
             Prevalence : 0.6536          
         Detection Rate : 0.5490          
   Detection Prevalence : 0.6863          
      Balanced Accuracy : 0.7219          
                                          
       'Positive' Class : neg             
                              

#### Grid

In [119]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)
grid <- expand.grid(size = seq(from = 1, to = 20, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

In [120]:
set.seed(seed_universal)
rna <- train(form = diabetes ~ .,data = treino, method = 'nnet',
             tuneGrid = grid, trControl = ctrl, MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Neural Network 

615 samples
  8 predictor
  2 classes: 'neg', 'pos' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 554, 554, 554, 554, 553, 553, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
   1    0.1    0.7270756  0.3270753
   1    0.2    0.7006875  0.2306773
   1    0.3    0.7512956  0.3863156
   1    0.4    0.7039133  0.2343904
   1    0.5    0.7126917  0.3347354
   1    0.6    0.7626917  0.4498852
   1    0.7    0.7365415  0.3657334
   1    0.8    0.7059757  0.2848042
   1    0.9    0.7367795  0.3572056
   2    0.1    0.6914067  0.2751845
   2    0.2    0.7433633  0.3901325
   2    0.3    0.7416711  0.4143130
   2    0.4    0.7203332  0.3155511
   2    0.5    0.7430724  0.4014418
   2    0.6    0.7302221  0.3745627
   2    0.7    0.7433104  0.3924946
   2    0.8    0.7741407  0.4688847
   2    0.9    0.7367266  0.3845488
   3    0.1    0.7218932  0.3473887
   3    0.2    0.7563458  0.4536825
   3    0.

In [121]:
predicoes.rna <- predict(rna, teste)

In [122]:
confusionMatrix(predicoes.rna, as.factor(teste$diabetes))

Confusion Matrix and Statistics

          Reference
Prediction neg pos
       neg  86  25
       pos  14  28
                                         
               Accuracy : 0.7451         
                 95% CI : (0.6684, 0.812)
    No Information Rate : 0.6536         
    P-Value [Acc > NIR] : 0.009661       
                                         
                  Kappa : 0.4082         
                                         
 Mcnemar's Test P-Value : 0.109315       
                                         
            Sensitivity : 0.8600         
            Specificity : 0.5283         
         Pos Pred Value : 0.7748         
         Neg Pred Value : 0.6667         
             Prevalence : 0.6536         
         Detection Rate : 0.5621         
   Detection Prevalence : 0.7255         
      Balanced Accuracy : 0.6942         
                                         
       'Positive' Class : neg            
                                         

### KNN

In [181]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))

In [182]:
set.seed(seed_universal)
knn <- train(diabetes ~ ., data = treino, method = 'knn', tuneGrid = tuneGrid)
knn

k-Nearest Neighbors 

615 samples
  8 predictor
  2 classes: 'neg', 'pos' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 615, 615, 615, 615, 615, 615, ... 
Resampling results across tuning parameters:

  k   Accuracy   Kappa    
   1  0.6731765  0.2890132
   3  0.6727623  0.2874482
   5  0.6873484  0.3101040
   7  0.7014126  0.3370444
   9  0.7123351  0.3565318
  11  0.7188705  0.3680130
  13  0.7275272  0.3864632
  15  0.7248063  0.3789608
  17  0.7225241  0.3699379
  19  0.7184907  0.3581286
  21  0.7190564  0.3572538

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 13.

In [183]:
predicoes.knn <- predict(knn, teste)
confusionMatrix(predicoes.knn, as.factor(teste$diabetes))

Confusion Matrix and Statistics

          Reference
Prediction neg pos
       neg  83  23
       pos  17  30
                                          
               Accuracy : 0.7386          
                 95% CI : (0.6615, 0.8062)
    No Information Rate : 0.6536          
    P-Value [Acc > NIR] : 0.01536         
                                          
                  Kappa : 0.4069          
                                          
 Mcnemar's Test P-Value : 0.42920         
                                          
            Sensitivity : 0.8300          
            Specificity : 0.5660          
         Pos Pred Value : 0.7830          
         Neg Pred Value : 0.6383          
             Prevalence : 0.6536          
         Detection Rate : 0.5425          
   Detection Prevalence : 0.6928          
      Balanced Accuracy : 0.6980          
                                          
       'Positive' Class : neg             
                              

---

## Banco (Classificação)

In [184]:
banco.dados <- read.csv('bases/banco/Material 02 - 11 – Banco - Dados.csv')

In [185]:
summary(banco.dados)

      age                  job         marital        education   default  
 Min.   :20.00   bluecollar  :59   divorced: 34   primary  : 40   no :292  
 1st Qu.:33.00   technician  :57   married :177   secondary:155   yes:  8  
 Median :38.00   management  :55   single  : 89   tertiary : 93            
 Mean   :40.65   admin.      :37                  unknown  : 12            
 3rd Qu.:48.00   services    :26                                           
 Max.   :78.00   selfemployed:16                                           
                 (Other)     :50                                           
    balance        housing    loan       y      
 Min.   :    0.0   no :133   no :245   no :262  
 1st Qu.:  175.5   yes:167   yes: 55   yes: 38  
 Median :  482.5                                
 Mean   : 1515.0                                
 3rd Qu.: 1653.5                                
 Max.   :16873.0                                
                                                

In [186]:
head(banco.dados)

age,job,marital,education,default,balance,housing,loan,y
30,unemployed,married,primary,no,1787,no,no,no
33,services,married,secondary,no,4789,yes,yes,no
35,management,single,tertiary,no,1350,yes,no,no
30,management,married,tertiary,no,1476,yes,yes,no
59,bluecollar,married,secondary,no,0,yes,no,no
35,management,single,tertiary,no,747,no,no,no


In [187]:
# Separa entre treino e teste

set.seed(seed_universal)
indices <- createDataPartition(banco.dados$y, p = 0.8, list = FALSE)
treino <- banco.dados[indices,]
teste <- banco.dados[-indices,]

### Redes Neurais

#### Hold out

In [127]:
set.seed(seed_universal)
rna <- train(y ~ ., data = treino, method = 'nnet', trace = FALSE)
rna

Neural Network 

241 samples
  8 predictor
  2 classes: 'no', 'yes' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 241, 241, 241, 241, 241, 241, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa        
  1     0e+00  0.8723151   0.0000000000
  1     1e-04  0.8723151   0.0000000000
  1     1e-01  0.8617192  -0.0005673441
  3     0e+00  0.8713425  -0.0018001533
  3     1e-04  0.8649918   0.0002290965
  3     1e-01  0.8418853   0.0280703226
  5     0e+00  0.8678320  -0.0033617713
  5     1e-04  0.8635611  -0.0097134785
  5     1e-01  0.8229977   0.0209140128

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 1 and decay = 1e-04.

In [128]:
predicoes.rna <- predict(rna, teste)

In [129]:
confusionMatrix(predicoes.rna, as.factor(teste$y))

Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  52   7
       yes  0   0
                                          
               Accuracy : 0.8814          
                 95% CI : (0.7707, 0.9509)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.59891         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.02334         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.8814          
         Neg Pred Value :    NaN          
             Prevalence : 0.8814          
         Detection Rate : 0.8814          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : no              
                                 

#### Cross validation

In [130]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)

In [131]:
set.seed(seed_universal)
rna <- train(y ~ ., data = treino, method = 'nnet', trace = FALSE, trControl = ctrl)
rna

Neural Network 

241 samples
  8 predictor
  2 classes: 'no', 'yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 217, 217, 217, 217, 217, 217, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa       
  1     0e+00  0.8715000   0.000000000
  1     1e-04  0.8715000   0.000000000
  1     1e-01  0.8715000   0.000000000
  3     0e+00  0.8715000   0.000000000
  3     1e-04  0.8756667   0.046666667
  3     1e-01  0.8508333   0.002051282
  5     0e+00  0.8675000  -0.006837607
  5     1e-04  0.8591667   0.010305250
  5     1e-01  0.8381667  -0.015555556

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 3 and decay = 1e-04.

In [132]:
predicoes.rna <- predict(rna, teste)

In [133]:
confusionMatrix(predicoes.rna, as.factor(teste$y))

Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  52   6
       yes  0   1
                                          
               Accuracy : 0.8983          
                 95% CI : (0.7917, 0.9618)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.44023         
                                          
                  Kappa : 0.2271          
                                          
 Mcnemar's Test P-Value : 0.04123         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.1429          
         Pos Pred Value : 0.8966          
         Neg Pred Value : 1.0000          
             Prevalence : 0.8814          
         Detection Rate : 0.8814          
   Detection Prevalence : 0.9831          
      Balanced Accuracy : 0.5714          
                                          
       'Positive' Class : no              
                                 

#### Grid

In [134]:
set.seed(seed_universal)
ctrl <- trainControl(method = 'cv', number = 10)
grid <- expand.grid(size = seq(from = 1, to = 20, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))

In [135]:
set.seed(seed_universal)
rna <- train(form = y ~ .,data = treino, method = 'nnet',
             tuneGrid = grid, trControl = ctrl, MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Neural Network 

241 samples
  8 predictor
  2 classes: 'no', 'yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 217, 217, 217, 217, 217, 217, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa        
   1    0.1    0.8756667   0.0466666667
   1    0.2    0.8715000   0.0000000000
   1    0.3    0.8715000   0.0000000000
   1    0.4    0.8715000   0.0000000000
   1    0.5    0.8715000   0.0000000000
   1    0.6    0.8715000   0.0000000000
   1    0.7    0.8715000   0.0000000000
   1    0.8    0.8715000   0.0000000000
   1    0.9    0.8715000   0.0000000000
   2    0.1    0.8590000   0.0111111111
   2    0.2    0.8631667  -0.0133333333
   2    0.3    0.8715000   0.0000000000
   2    0.4    0.8715000   0.0000000000
   2    0.5    0.8715000   0.0000000000
   2    0.6    0.8715000   0.0000000000
   2    0.7    0.8715000   0.0000000000
   2    0.8    0.8715000   0.0000000000
   2    0.9    0.8715000   0.0000000000
   3   

In [136]:
predicoes.rna <- predict(rna, teste)

In [137]:
confusionMatrix(predicoes.rna, as.factor(teste$y))

Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  52   7
       yes  0   0
                                          
               Accuracy : 0.8814          
                 95% CI : (0.7707, 0.9509)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.59891         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.02334         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.8814          
         Neg Pred Value :    NaN          
             Prevalence : 0.8814          
         Detection Rate : 0.8814          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : no              
                                 

### KNN

In [196]:
tuneGrid <- expand.grid(k = c(1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21))

In [197]:
set.seed(seed_universal)
knn <- train(y ~ ., data = treino, method = 'knn', tuneGrid = tuneGrid)
knn

k-Nearest Neighbors 

241 samples
  8 predictor
  2 classes: 'no', 'yes' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 241, 241, 241, 241, 241, 241, ... 
Resampling results across tuning parameters:

  k   Accuracy   Kappa        
   1  0.7984854   0.1182560837
   3  0.8061770   0.0503916113
   5  0.8294749   0.0266087960
   7  0.8446598   0.0343770628
   9  0.8484313   0.0182649861
  11  0.8586352  -0.0060930345
  13  0.8649566   0.0087386176
  15  0.8677743   0.0007374047
  17  0.8672814  -0.0028290445
  19  0.8673246  -0.0031148978
  21  0.8678491  -0.0010977128

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 21.

In [198]:
predicoes.knn <- predict(knn, teste)
confusionMatrix(predicoes.knn, as.factor(teste$y))

Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  52   7
       yes  0   0
                                          
               Accuracy : 0.8814          
                 95% CI : (0.7707, 0.9509)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.59891         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.02334         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.8814          
         Neg Pred Value :    NaN          
             Prevalence : 0.8814          
         Detection Rate : 0.8814          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : no              
                                 

---